In [ ]:
!pip install git+https://github.com/facebookresearch/detectron2@main
!pip install git+https://github.com/rwightman/pytorch-image-models@main

In [ ]:
!git clone https://github.com/iKrishneel/detectron2_timm.git
%cd /content/detectron2_timm/
!git pull origin master
!pip install -e .

In [ ]:
!wget https://dl.fbaipublicfiles.com/xcit/coco/maskrcnn_xcit_small_12_p8.pth

In [ ]:
!python ./tools/weight_converter.py --name mrcnn_xcit_small_12_p8 --weights ./maskrcnn_xcit_small_12_p8.pth --output_dir ./pretrained/

In [1]:
import os.path as osp
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

import torch
from torchvision.datasets import CocoDetection

from detectron2.engine.defaults import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

from detectron2_timm.config import get_cfg

In [ ]:
"""
root = '/workspace/research/datasets/coco/'
dataset = CocoDetection(
    osp.join(root, 'val2017'), 
    annFile=osp.join(root, 'annotations/instances_val2017.json')
)
"""

folder_name = '../config/'
cfg = get_cfg()
cfg.merge_from_file('./config/xcit/xcit_small_12_p8_224_fpn.yaml')
cfg.MODEL.DEVICE = "cpu"
cfg.MODEL.WEIGHTS = './pretrained/mrcnn_xcit_small_12_p8.pth'

predictor = DefaultPredictor(cfg)
metadata = MetadataCatalog.get(cfg.DATASETS.TEST[0])

In [ ]:
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 80 

In [ ]:
def test_one(image, threshold=0.5):
    visualizer = Visualizer(image, metadata=metadata, scale=1.0)
    
    r = predictor(image)
    instances = r['instances'].to('cpu').get_fields()
   
    scores = instances['scores'].numpy()
    remove_indices = scores < threshold
    
    scores = np.delete(scores, remove_indices, 0)
    bboxes = np.delete(instances['pred_boxes'].tensor.numpy(), remove_indices, 0)
    labels = np.delete(instances['pred_classes'].numpy(), remove_indices, 0)
    
    try:
        masks = np.delete(instances['pred_masks'].numpy(), remove_indices, 0)
    except KeyError:
        masks = None

    labels = np.array(metadata.get('thing_classes'))[labels]
    viz = visualizer.overlay_instances(labels=labels, boxes=bboxes, masks=masks)
    
    image = viz.get_image()

    plt.axis('off')
    plt.imshow(image)
    plt.show()

In [ ]:
!wget http://injapan.gaijinpot.com/wp-content/uploads/2013/06/bus.jpg -O bus.jpg

im = cv.imread('bus.jpg')
test_one(im)